# Dense Document Retriever - Experimento

Ranqueia textos de acordo com um outro texto de entrada através do ranqueador DPR do hugginfaces.<br>

### **Em caso de dúvidas, consulte os [tutoriais da PlatIAgro](https://platiagro.github.io/tutorials/).**

## Declaração de parâmetros e hiperparâmetros

Declare parâmetros com o botão  na barra de ferramentas.<br>
A variável `dataset` possui o caminho para leitura do arquivos importados na tarefa de "Upload de dados".<br>
Você também pode importar arquivos com o botão  na barra de ferramentas.

In [3]:
# dataset = "/tmp/data/paracrawl_en_pt_test.csv" #@param {type:"string"}
dataset = "/tmp/data/reports_contexts-5.csv" #@param {type:"string"}
column = "context" #@param {type:"string",label:"Coluna do dataframe de entrada com os contextos para ranquear"}
question = "What is the best weed herbicide?" #@param {type:"string",label:"Pergunta para a qual os contextos devem ser ranqueados"}

#Hyperparams
top = 10 #@param {type:"integer",label:"Quantidade de contextos para retornados"}
inner_batch_size = 10 #@param {type:"integer",label:"Batch size",description:"Tamanho do batchsize interno ao DPR do hugginfaces"}
tokenizer_fn = "facebook/dpr-reader-single-nq-base" #@param ["facebook/dpr-reader-single-nq-base"] {type:"string",label:"Nome do tokenizador",description:"Nome do tokenizador na base do hugginfaces"}
tokenizer_max_len = 512 #@param {type:"integer",label:"Tamanho máximo das sentenças",description:" Tamanho máximo das sentenças do tokenizador do hugginfaces"}
dpr_fn = "facebook/dpr-reader-single-nq-base" #@param ["facebook/dpr-reader-single-nq-base"] {type:"string",label:"Nome do modelo",description:"Nome do modelo na base do hugginfaces"}



# Acesso ao conjunto de dados¶
O conjunto de dados utilizado nesta etapa será o mesmo carregado através da plataforma.
O tipo da variável retornada depende do arquivo de origem:

* pandas.DataFrame para CSV e compressed CSV: .csv .csv.zip .csv.gz .csv.bz2 .csv.xz
* Binary IO stream para outros tipos de arquivo: .jpg .wav .zip .h5 .parquet etc

In [4]:
import pandas as pd

df = pd.read_csv(dataset)

## Funções de apoio

In [5]:
def build_result_dataframe(sim_contexts_ids,scores,contexts):
    sim_contexts = [contexts[i] for i in sim_contexts_ids[0]]
    df = pd.DataFrame({'doc_id':sim_contexts_ids[0],'score':scores[0],'context':sim_contexts})
    df = df.sort_values(by=['score'], ascending=False).reset_index(drop=True)
    return df

## Conteúdo da tarefa

In [6]:
from model_dense_retriever import EnglishDPRRetriever

In [7]:
retriever = EnglishDPRRetriever(tokenizer_fn = tokenizer_fn,
                                dpr_fn = dpr_fn,
                                tokenizer_max_len = tokenizer_max_len)
report_contexts = list(df[column])

Downloading:   0%|          | 0.00/484 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [8]:
sim_contexts_ids, scores = retriever(questions=question,
                                     passages = report_contexts,
                                     inner_batch_size = inner_batch_size,
                                     top=top)

Token indices sequence length is longer than the specified maximum sequence length for this model (518 > 512). Running this sequence through the model will result in indexing errors


In [9]:
df = build_result_dataframe(sim_contexts_ids,scores,report_contexts)
df.head(10)

,doc_id,score,sim_contexts
0,684,-2.746182,"7. REFERÊNCIAS BRIDGES, D.C. Benefits of Tr..."
1,779,-3.493546,"1.3 Palavras-Chave: fomesafen, lactofen, bent..."
2,207,-4.353143,"7. REFERÊNCIAS BARROSO, A.L. de L.; FERREIR..."
3,542,-4.367135,6. CONCLUSÕES Nas condições em que o projet...
4,157,-4.596657,"7. REFERÊNCIAS BARROSO, A.L. de L.; FERREIR..."
5,835,-4.642037,"7. REFERÊNCIAS BARELA, J.F. Seletividade de ..."
6,741,-5.071363,"7. REFERÊNCIAS BARROSO, A.L. de L.; FERREIR..."
7,869,-5.251039,5.5 ORS Agile
8,1333,-5.699102,"7. REFERÊNCIAS BOLLMAN, J.D.; BOERBOOM, C.M..."
9,1374,-5.825028,"1.3 Palavras-Chave: mistura formulada, fitoto..."


In [ ]:
df.to_csv(dataset, index=False)

## Salva resultados da tarefa

A plataforma guarda o conteúdo de `/tmp/data/` para as tarefas subsequentes.<br>
Use essa pasta para salvar modelos, metadados e outros resultados.

In [10]:
from joblib import dump

artifacts = {
    "model":retriever,
    "report_contexts":report_contexts,
    "inner_batch_size":inner_batch_size,
    "top":top,
}

dump(artifacts, "/tmp/data/sparse_retriever.joblib")

['/tmp/data/sparse_retriever.joblib']